In [2]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")

#import src
import Cleanup
import Transformation
import FileProcessing

original File path: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\scripts
Data File path: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\data


In [3]:
#define the objects to be called later. 
cleanup=Cleanup.Cleanup()
transform=Transformation.Transformation()
fileProcessing=FileProcessing.FileProcessing()

Read the File from Data folder and remove all duplicates. 

In [4]:
file_name='Next_Generation_Simulation__NGSIM__Vehicle_Trajectories_and_Supporting_Data'
ngsim=fileProcessing.read_input(file_name)

File Read Complete: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\data/Next_Generation_Simulation__NGSIM__Vehicle_Trajectories_and_Supporting_Data.csv


Remove any Duplicate records from the original File. 

In [5]:
cleanup.remove_dups(ngsim)

704000 duplicate values have been removed


,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Width,...,D_Zone,Int_ID,Section_ID,Direction,Movement,Preceding,Following,Space_Headway,Time_Headway,Location
0,515,2330,1123,1118848075000,30.034,188.062,6451203.729,1873252.549,13.0,6.9,...,NaN,NaN,NaN,NaN,NaN,500,523,119.10,5.11,us-101
2,2224,6548,1902,1113437421700,41.429,472.901,6042814.264,2133542.012,14.3,6.9,...,NaN,NaN,NaN,NaN,NaN,2208,2211,53.34,2.01,i-80
3,2127,6459,567,1118847624800,19.632,1775.614,6452425.122,1872172.475,13.5,6.9,...,NaN,NaN,NaN,NaN,NaN,2124,2132,48.92,1.30,us-101
4,1033,4827,592,1118848324700,6.202,1701.144,6452347.673,1872258.452,13.5,4.4,...,NaN,NaN,NaN,NaN,NaN,1029,1040,38.81,0.92,us-101
6,1890,9157,628,1118849672700,53.514,817.521,6451655.238,1872800.663,24.0,8.5,...,NaN,NaN,NaN,NaN,NaN,1882,1897,102.65,2.27,us-101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11850521,1355,5622,938,1113438127100,18.275,1464.336,6042648.149,2134520.896,12.3,6.8,...,NaN,NaN,NaN,NaN,NaN,1351,1361,26.91,15.92,i-80
11850522,1474,5688,401,1113438133700,5.509,560.412,6042767.840,2133624.549,17.8,7.9,...,NaN,NaN,NaN,NaN,NaN,1468,1480,74.49,1.73,i-80
11850523,398,2368,654,1113437801700,18.903,635.841,6042771.741,2133701.076,15.2,8.5,...,NaN,NaN,NaN,NaN,NaN,385,406,90.95,3.77,i-80
11850524,599,875,577,1113436854400,77.094,621.358,6042831.282,2133693.854,15.3,6.4,...,NaN,NaN,NaN,NaN,NaN,0,611,0.00,0.00,i-80


Feet to Metres:

Below Four parametrs consider distance in Feet and they are converted to metre. Speed to Metre/ second from Feet/second and acceleration from feet/second Square to Metre/Second Square. 

Drop the columns we wont use in the model.
Add Relative time which is the time from the first timframe in the dataset. This is calculated by using the minimum value ofGlobal time(absolute time from 1970)

In [6]:
ngsim=transform.convert_feet_to_metre(ngsim)

In [7]:
ngsim.shape

(11146526, 25)

In [8]:
ngsim=transform.drop_not_required_columns(ngsim)

Create Placeholders for the user defined columns we will be populating later, to be used in the model. 
Map the Vehicle classes to their english equivalent. 
Filter the original file into US-101 and I-80 to perform and populate the required fields later. These files contain same values of Vehicle ID used for multiple vehicles based on the highway. 



In [9]:
ngsim=transform.create_column_placeholders(ngsim)

Split data into US-101 and I-80 specific dataframes.

In [10]:
filtered_ngsim_U,filtered_ngsim_I = transform.bifurcate_highways(ngsim)

Create Sets to use and create the L-F Pairs. 

In [11]:
v_Class_M_U,v_Class_C_U,v_Class_HV_U = transform.class_vehicle_sets(filtered_ngsim_U)
v_Class_M_I,v_Class_C_I,v_Class_HV_I = transform.class_vehicle_sets(filtered_ngsim_I)


Add the Preceding Vehicle Class to the new previous class column

In [12]:
filtered_ngsim_U=transform.preceding_vehicle_class(filtered_ngsim_U,v_Class_M_U,v_Class_C_U,v_Class_HV_U)
filtered_ngsim_I=transform.preceding_vehicle_class(filtered_ngsim_I,v_Class_M_I,v_Class_C_I,v_Class_HV_I)

Map the Vehicle Length of the previous vehicle into the row. 

In [13]:
filtered_ngsim_U=transform.preceding_vehicle_length(filtered_ngsim_U)
filtered_ngsim_I=transform.preceding_vehicle_length(filtered_ngsim_I)

Convert the Front to Front Space Headway to Back Bumper (preceding Vehicle) to front Bumper(of Following ) Headway. This will give the actual distance between the vehicle ignoring the vehicle length.
Find the time it would take for the Following vehicle to reach the rear Bumper of the Preceding Vehicle. 

In [14]:
filtered_ngsim_U= transform.front_to_front_bumper_details_changed_to_rear_to_front_bumper_details(filtered_ngsim_U)
filtered_ngsim_I= transform.front_to_front_bumper_details_changed_to_rear_to_front_bumper_details(filtered_ngsim_I)

Vehicle Combination of the Preceding and the Following Vehicle. Populate the Previous vehicle details with Subject vehicle Details

In [15]:
filtered_ngsim_U=transform.map_preceding_vehicle_details(filtered_ngsim_U)
filtered_ngsim_I=transform.map_preceding_vehicle_details(filtered_ngsim_I)
filtered_ngsim_U=transform.time_pairs(filtered_ngsim_U)
filtered_ngsim_I=transform.time_pairs(filtered_ngsim_I)

Filter the vehicles based on the filteration criterias:

- Remove all Vehicles which have bad Vehicle Length and Type. i.e. same vehicle having two vehicle Types in different pair information.
- Remove first 5 seconds for a Pair when Lead Vehicle is overtaking. Last 5 from the pair which the vehicle is leaving(same vehicle was Subject in that scneario).
- Remove Lane 4 and above as they are shoulders/ramps and exits. 
- Remove trajectories which are less than 30 seconds. 


In [16]:
filtered_ngsim_U=cleanup.filter_records_for_model(filtered_ngsim_U)
filtered_ngsim_I=cleanup.filter_records_for_model(filtered_ngsim_I)

dataset before Row Removal(4098933, 35)
us-101: 154876 vehicles first and Last 5 seconds removed from Lane 1 due to lane changing
us-101: 194882 vehicles first and Last 5 seconds removed from Lane 2 due to lane changing
us-101: 195007 vehicles first and Last 5 seconds removed from Lane 3 due to lane changing
us-101: 530853 have multiple lengths and classes for same Vehicle ID
us-101: 780619 have total car following less than 30 seconds
us-101: 137686 have time Headway less than 5 seconds
us-101: 1598920 have Lane ID as 7 or 8 which are the Ramps
us-101: 2521752 rows removed using above criterias
dataset before Row Removal(4566387, 35)
i-80: 153820 vehicles first and Last 5 seconds removed from Lane 1 due to lane changing
i-80: 157225 vehicles first and Last 5 seconds removed from Lane 2 due to lane changing
i-80: 163719 vehicles first and Last 5 seconds removed from Lane 3 due to lane changing
i-80: 830630 have multiple lengths and classes for same Vehicle ID
i-80: 940574 have total ca


1. Velocity difference of the Lead minus Preceding Vehicle.
2. Acceleration difference of the Lead minus Preceding Vehicle.
3. Find the Pair details of the Lead_following 
4. Find the cumulative time the Pair runs within the highway as a pair. 

In [17]:

filtered_ngsim_U=transform.remap_pair_time(filtered_ngsim_U)
filtered_ngsim_I=transform.remap_pair_time(filtered_ngsim_I)

Merge the I 80 and US-110 Highway cleaned files. 

In [18]:
cleaned_NGSIM= fileProcessing.merge_files(filtered_ngsim_U , filtered_ngsim_I)


 Merged Record Count:2569188, df1:1577181, df2:992007


Export the Cleaned NGSIM File- The below can be used in case Cleaned Data is required. 

In [19]:
fileProcessing.export_file(cleaned_NGSIM,'Cleaned_NGSIM_Data')

True

Create the Sample Trajectory Pairs to view the results of the Car Following Model. 
3 Pair for Each Type of Pairs that remain in the Cleaned File. 

In [20]:
x=cleaned_NGSIM.groupby(['Vehicle_combination','L-F_Pair'])['total_pair_duration'].max().sort_values(ascending=False)
x

Vehicle_combination  L-F_Pair 
Car-Car              2054-2057    366.8
                     1841-1848    294.9
                     1881-1889    244.2
                     1668-1676    232.2
                     1839-1849    229.6
                                  ...  
                     1032-1038     30.0
                     1436-1442     30.0
                     1216-1221     30.0
                     3215-3225     30.0
                     1103-1111     30.0
Name: total_pair_duration, Length: 4806, dtype: float64

In [21]:
x_hv_car=x[x.index.get_level_values('Vehicle_combination').isin(['Heavy Vehicle-Car'])]
hv_car_pair= x_hv_car[x_hv_car>60].tail(3).index.get_level_values('L-F_Pair').values
hv_car_pair


array(['2695-2725', '3084-3094', '551-560'], dtype=object)

In [22]:
x_car_car=x[x.index.get_level_values('Vehicle_combination').isin(['Car-Car'])]
Car_car_pair= x_car_car[x_car_car>60].tail(3).index.get_level_values('L-F_Pair').values
Car_car_pair


array(['1304-1309', '439-444', '1635-1642'], dtype=object)

In [23]:
x_car_hv=x[x.index.get_level_values('Vehicle_combination').isin(['Car-Heavy Vehicle'])]
car_hv_pair= x_car_hv[x_car_hv>60].tail(3).index.get_level_values('L-F_Pair').values
car_hv_pair


array(['2322-2330', '2785-2804', '2725-2717'], dtype=object)

In [24]:
trajectory_pairs=cleaned_NGSIM[cleaned_NGSIM['L-F_Pair'].isin(car_hv_pair )|cleaned_NGSIM['L-F_Pair'].isin(Car_car_pair)|cleaned_NGSIM['L-F_Pair'].isin(hv_car_pair)]

In [25]:
trajectory_pairs

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,lane_changes,preceding_Vehicle_Velocity,preceding_Vehicle_Acceleration,preceding_car_lane_changes,preceding_Local_Y,preceding_v_Class,jerk,total_pair_duration,pair_Time_Duration,total_pair_dur
916994,2330,8430,1118848685000,8.434121,75.145087,6.70560,3,9.140952,0.000000,3,...,False,7.604760,0.000000,True,91.089175,2.0,0.00000,66.8,0.0,56.7
917061,2330,8431,1118848685100,8.458200,76.059182,6.70560,3,9.140952,0.000000,3,...,False,7.601712,-0.012192,True,91.849651,2.0,-7.98576,66.8,0.1,56.7
917077,2330,8432,1118848685200,8.482279,76.973278,6.70560,3,9.140952,0.000000,3,...,False,7.601712,-0.018288,True,92.609822,2.0,-16.15440,66.8,0.2,56.7
917120,2330,8433,1118848685300,8.506358,77.887373,6.70560,3,9.140952,0.000000,3,...,False,7.607808,0.112776,True,93.369079,2.0,0.00000,66.8,0.3,56.7
917194,2330,8434,1118848685400,8.530133,78.800858,6.70560,3,9.147048,0.121920,3,...,False,7.635240,0.371856,True,94.128641,2.0,-7.22376,66.8,0.4,56.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813958,1642,6425,1113438207400,4.816450,230.488236,4.05384,2,4.428744,1.874520,2,...,True,7.028688,3.413760,True,247.469558,2.0,18.74520,60.1,24.6,25.0
813998,1642,6426,1113438207500,4.818278,230.950008,4.05384,2,4.556760,0.509016,2,...,True,7.412736,3.084576,True,248.191020,2.0,5.09016,60.1,24.7,25.0
814052,1642,6427,1113438207600,4.819802,231.412999,4.05384,2,4.587240,-0.112776,2,...,True,7.632192,1.161288,True,248.960640,2.0,-1.18872,60.1,24.8,25.0
814097,1642,6428,1113438207700,4.821326,231.871114,4.05384,2,4.578096,-0.076200,2,...,True,7.705344,0.021336,True,249.738490,2.0,-0.76200,60.1,24.9,25.0


In [26]:
fileProcessing.export_file(trajectory_pairs,'Plot_Pairs_NGSIM_Data')

True

In [27]:
trajectory_removed_NGSIM = cleaned_NGSIM.drop(trajectory_pairs.index,axis=0)

Split the Data into Test/Validate/Train in a ratio of 70/0/30

In [33]:
train_df, val_df, test_df=transform.train_test_pair(trajectory_removed_NGSIM,0.9,True)

In [29]:
fileProcessing.export_file(train_df,'train_df_cleaned_NGSIM')
fileProcessing.export_file(val_df,'val_df_cleaned_NGSIM')
fileProcessing.export_file(test_df,'test_df_cleaned_NGSIM')

True

In [36]:
train_df_cnt=train_df.shape[0]
val_df_cnt=val_df.shape[0]
test_df_cnt=test_df.shape[0]
total_cnt=test_df_cnt+val_df_cnt+train_df_cnt

print(f"train df Count:{train_df_cnt}, percentage: {train_df_cnt/total_cnt}")
print(f"val df Count:{val_df_cnt}, percentage: {val_df_cnt/total_cnt}")
print(f"test df Count:{test_df_cnt}, percentage: {test_df_cnt/total_cnt}")

train df Count:2343692, percentage: 0.8307064342840453
val df Count:217250, percentage: 0.07700285397919558
test df Count:260382, percentage: 0.09229071173675905
